In [14]:
import pandas as pd
from utils.config import domainConfig, domainConfigClass, supplierId
from typing import List
from collections import Counter
from datetime import datetime


def generateInvoiceId(date: datetime, locationCode: str, invoiceVersion: int):
    return f'{date.strftime("%d%m%Y")}U{locationCode}{invoiceVersion}'


def generatePONo(date: datetime, storeId: str, supplierId: str):
    return f'{date.strftime("%Y%m%d")}-{storeId}-{supplierId}'


def nameExtracter(rightNamesList: List[str], wrongName: str) -> str:
    chances = [0] * len(rightNamesList)

    counterOriginal = Counter(wrongName.lower())
    for i, rightName in enumerate(rightNamesList):
        counterLocation = Counter(rightName.lower())
        chances[i] = (
            sum((counterOriginal & counterLocation).values())
            * 2
            / (sum(counterOriginal.values()) + sum(counterLocation.values()))
        )
    return rightNamesList[chances.index(max(chances))]
    return rightNamesList[chances.index(max(chances))]

In [2]:
df = pd.read_excel("file.xlsx", sheet_name="Sheet1")

In [3]:
dropedDf = df.dropna(how="all")[
    [
        column.rawSheet.columnName
        for column in domainConfigClass.columns
        if column.rawSheet is not None
    ]
]

In [4]:
dropedDf.columns = [
    column.invoicePdf.columnName
    for column in domainConfigClass.columns
    if column.rawSheet is not None
]

In [ ]:
pdfColumns = list(
    map(
        lambda x: x.columnName,
        sorted(
            [
                column.invoicePdf
                for column in domainConfigClass.columns
                if column.invoicePdf is not None and column.invoicePdf.index is not None
            ],
            key=lambda x: x.index,
        ),
    )
)

extraColumns = [
    column.invoicePdf.columnName
    for column in domainConfigClass.columns
    if column.rawSheet is None and column.invoicePdf is not None
]

date = datetime.strptime(str(df["Date"][0]), "%Y-%m-%d 00:00:00")
invoiceVersion = 1

pdfDF = dropedDf.copy()

for column in extraColumns:
    pdfDF[column] = ""

# pdfDF = pdfDF[pdfColumns]

pdfDF["Dispatched Qty"] = pdfDF["Dispatched Qty"].astype(int)
pdfDF["Rate"] = pdfDF["Rate"].astype(int)

pdfDF["Total Amount"] = pdfDF["Dispatched Qty"] * pdfDF["Rate"]

pdfDF["Location"] = pdfDF["Location"].apply(
    lambda x: nameExtracter(
        [location.locationName for location in domainConfigClass.locations], x
    )
)

In [22]:
tableDf = {
    location.locationName: {
        "dataFrame": pdfDF[pdfDF["Location"] == location.locationName][
            pdfColumns
        ].reset_index(drop=True),
        "invoiceNumber": generateInvoiceId(date, location.code, invoiceVersion),
        "poNo": generatePONo(date, location.storeId, supplierId),
        "shippingAddress": location.shippingAddress,
        "retailer": location.retailer,
    }
    for location in domainConfigClass.locations
}

In [23]:
tableDf["Ayodhya Nagar"]["retailer"]

'Rajidi Retail Pvt Ltd'